# Book Recommendations

One of the solutions for my business problem is to recommend books to customers. With the help of the recommendations;

- Customers do not need to spend to much time to find suitable book.
- When they find books easily, they will buy more books.
- Also, they will advice this online store and give good reviews to it. It means more customer and more sales. 
- Also, when customers spend less time on server, it decreases the technical problems on website. 

### Aim of This Notebook:

My aim in this notebook to recommend books to customers in 2 different ways.

In [1]:
# dataframe and series 
import pandas as pd
import numpy as np
import scipy
import math

# sklearn imports 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,balanced_accuracy_score
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics.pairwise as pw
from sklearn.metrics.pairwise import pairwise_distances

# To plot
import matplotlib.pyplot as plt  
%matplotlib inline    
import matplotlib as mpl
import seaborn as sns

import random
from scipy import sparse
from scipy.stats import pearsonr
import re

In [2]:
pd.options.display.max_columns=100 # To see the hidden columns in dataframe

In [3]:
df = pd.read_csv('cleaned_data.csv') # taking whole data

My data is too big to take pivot table in my basic computer and impossible to work on it. So, I will take around top 1000 users and 1000 books and do my system according to them.

## Choosing 1000 users - 1000 books

Firstly, I would like to see users review numbers and books review numbers.

In [4]:
# creting new column which consists of reviewer numbers of user
df['Reviewer_Count'] = df['reviewerID'].map(df['reviewerID'].value_counts())

In [5]:
# creating new column for review numbers of books
df['Book_Count'] = df['asin'].map(df['asin'].value_counts())

In [6]:
df = df.sort_values(by=['Reviewer_Count'],ascending=False)

In [7]:
df.reset_index(drop=True,inplace=True) 

In [8]:
df.isna().sum() # to check title column

overall               0
verified              0
reviewTime            0
reviewerID            0
asin                  0
style                 0
reviewText            0
summary            1716
title             15957
day                   0
month                 0
year                  0
sentiment             0
review_clean        242
Reviewer_Count        0
Book_Count            0
dtype: int64

I want to give information with book titles, so I will drop the reviews which do not have titles. In this data, 'asin' shows the book ID. If the title is not found in meta data, it is shown as null. So, I will drop them to see titles.

In [9]:
df.dropna(subset=['title'], inplace=True) # dropping without title columns

# Taking Samples 

I will have 2 sampled data to compare recommandation results at the end. In this notebook, I will use df_1000 which contains approximately 1000 user and 1000 books.

In [10]:
df_100 = df.loc[df['Reviewer_Count']>150] 

In [11]:
df_1000 = df_100.loc[df_100['Book_Count']>150]

In [12]:
df_1000['reviewerID'].nunique()

1096

In [13]:
df_1000['asin'].nunique()

1135

In above cells, I select users who have more than 150 reviews. Also between them, I select books which have more than 150 reviews.

In [366]:
df_200=df.loc[df['Reviewer_Count']>100]

In [371]:
df_2000 = df_200.loc[df_200['Book_Count']>100]

I also take more than 100 reviews for each same way to keep more big sample.

In [359]:
df_1000.to_csv('df_1000.csv',index = False) # writing to csv for later use

In [372]:
df_2000.to_csv('df_2000.csv',index = False) # writing to csv for later use

# Two Main Recommendation Systems

## Content-based Filtering

<img src="images/1_BME1JjIlBEAI9BV5pOO5Mg.png" style='width:300px; height:400px'/>


*image source: https://towardsdatascience.com/how-to-build-from-scratch-a-content-based-movie-recommender-with-natural-language-processing-25ad400eb243*

In content-based filtering, system do not care about what other people like. It considers what the customer like and find similar items to recommend.

## Collaborative Filtering

<img src="images/1_x8gTiprhLs7zflmEn1UjAQ.png" style='width:400px; height:400px'/>

*image source: https://towardsdatascience.com/how-to-build-from-scratch-a-content-based-movie-recommender-with-natural-language-processing-25ad400eb243*

There are many ways to advice products to customers, in this project I will try collaborative filtering technique with latent factor models as Singular Value Decomposition (SVD). Collaborative filtering makes predictions about the previous likes of customers such as if customer A and B have same likes/dislikes on items, this method recommend books according to similarities of these people instead of doing this randomly. Collaborative filtering details can be found [here](https://en.wikipedia.org/wiki/Collaborative_filtering).

SVD is a factorization of matrix that generalizes the eigendecomposition of a square normal matrix to any matrix via an extension of the polar decomposition. Details about SVD can be found [here](https://en.wikipedia.org/wiki/Singular_value_decomposition).

Latent factor models compress user-item matrix so the advantage of this model is working with small matrix in lower-dimensional space. 

![title](images/1_7uW5hLXztSu_FOmZOWpB6g.png)

*image source: https://towardsdatascience.com/various-implementations-of-collaborative-filtering-100385c6dfe0*

In [14]:
# taking pivot table of user-item

pivot = pd.pivot_table(df_1000, index='reviewerID', columns=['asin'], values='overall').fillna(0)

pivot.head(5)

asin,B003FL6X52,B003JTHMO6,B003P2QCE8,B003XVYJ2S,B0043GX2FW,B00457VKIA,B004A8ZT9A,B004AYDJM8,B004C44556,B004D9FGD0,B004JN05KY,B004KKY782,B004KZPK6K,B004LB4FBE,B004M18ZRS,B004OA5ZTS,B004RVNMBW,B004RZ2660,B004SI455Q,B004UHLJFO,B004UJTG6Q,B004V49BLU,B004VF69LY,B004W8D9V8,B004X6R8Y8,B004XT6L8E,B004XW3LLQ,B004Y51PW4,B004Z99U3K,B0051EZBRM,B0054H1I7S,B0057G5BQ0,B0057QO8Q4,B00586XP5S,B0058OJ8AG,B005C2COA0,B005C5YZ86,B005CCM48M,B005COO1X6,B005D75Z8C,B005FIN9YG,B005G7ZFXY,B005GI9EXU,B005GSS26K,B005IG6488,B005K4W0QS,B005LC1NGW,B005ME39HU,B005MGEO36,B005MT03FQ,...,B01EIPX54U,B01EJ2BV0W,B01EKIELGG,B01EMF4FXG,B01ER3O8UI,B01EZ9ZK7O,B01F2EXCF8,B01F2WTGFK,B01FB5MIA8,B01FBIC51W,B01FCBU7OU,B01FD1Q27U,B01FIGE8YE,B01FK7LI7G,B01FKDKB96,B01FLDUVIG,B01FRK7VO0,B01FRQ4Z8E,B01FSDI7H6,B01FTEBE74,B01G264272,B01G3UFLMM,B01G7Q908I,B01GBYTK36,B01GD3CF5U,B01GEKTEY2,B01GF9OSI4,B01GGLV2IU,B01GGNXFEW,B01GJRQKBK,B01GKU3D1G,B01GM4MH6C,B01GPEDC3G,B01GQE49E6,B01GQW4UQ0,B01GT44VLO,B01GT4NXU4,B01GW1BZBI,B01GYPQQRU,B01GZ070A6,B01GZ7IMJC,B01H0UI0QS,B01H4NVSOC,B01H5MF0JG,B01H62L5GW,B01H8JWYIQ,B01H941TYK,B01HDF9IKM,B01HDNEFGQ,B01HF6OU70
reviewerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A0020356UF96ZV361ST,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A00463782V7TKAP9EMNL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0099735VDZ3HDCAAYKL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A103KWX1QFCG6I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A103M7E0BSFC83,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# changing pivot table to matrix
pivot_mat = pivot.as_matrix()

pivot_mat[:5]

/Users/ezgi/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
# My reviewer id column is index, so I turned it to list
reviewer_id = list(pivot.index)
reviewer_id[:10]

['A0020356UF96ZV361ST',
 'A00463782V7TKAP9EMNL',
 'A0099735VDZ3HDCAAYKL',
 'A103KWX1QFCG6I',
 'A103M7E0BSFC83',
 'A1098Z3D7ENJ2F',
 'A10DX94ATZQEFG',
 'A10ED33F8VY4GU',
 'A10JAUCIGVRW9F',
 'A10PEXB6XAQ5XF']

In [17]:
# Sparcing pivot matrix 
sparse_matrix = csr_matrix(pivot_mat)

sparse_matrix

<1096x1135 sparse matrix of type '<class 'numpy.float64'>'
	with 26988 stored elements in Compressed Sparse Row format>

I created sparce matrix because lots of the values in pivot matrix is zero (if user have no rating on that book, it is shown as zero). So, different than dense matrix which stores all values, sparse matrix keeps non-zero values according to row and column indices.

## Matrix-Factorization

![title](images/1_2i-GJO7JX0Yz6498jUvhEg.png)
*image source: https://towardsdatascience.com/various-implementations-of-collaborative-filtering-100385c6dfe0*

In [18]:
# factor numbers of the user-item matrix

factor_n = 15

# matrix factorization of the user-item matrix

U, sigma, V = svds(sparse_matrix, k = factor_n) #defining elements of the factorization

In [19]:
# to see and check the dimensions of matrix
print(U.shape)
print(V.shape)
sigma = np.diag(sigma)
print(sigma.shape)

(1096, 15)
(15, 1135)
(15, 15)


In [20]:
# dot product of matrix
pred_rating = np.dot(np.dot(U,sigma),V)

pred_rating[:5]

array([[ 1.10239408e-03,  1.34736759e-02, -5.86820635e-05, ...,
        -2.68693144e-02, -2.01760244e-02, -4.75279090e-02],
       [ 3.34235072e-03,  4.03884159e-03, -1.15664696e-05, ...,
        -1.37850662e-03, -2.61606703e-03, -1.18982518e-03],
       [ 9.22264322e-02, -5.63993179e-02,  2.54544712e-04, ...,
         1.00024480e-01,  7.52609550e-02, -4.05629957e-01],
       [ 5.35298052e-02, -1.75656540e-02,  1.52259135e-04, ...,
         5.15084647e-02,  1.08079432e-01,  5.16179317e-02],
       [-2.81642346e-03,  6.93704762e-02,  4.88292424e-04, ...,
         2.53128868e-02, -1.04887572e-02, -6.69418895e-02]])

I need to normalize my matrix before recommending because of zero values.

In [21]:
# normalizing matrix
pred_rating_n = (pred_rating - pred_rating.min()) / (pred_rating.max() - pred_rating.min())

In [22]:
pred_rating_n[:5]

array([[0.24079927, 0.24213786, 0.24067364, ..., 0.23777268, 0.2384969 ,
        0.23553738],
       [0.24104164, 0.241117  , 0.24067874, ..., 0.24053083, 0.24039692,
        0.24055125],
       [0.25065906, 0.23457747, 0.24070753, ..., 0.25150282, 0.24882336,
        0.19679006],
       [0.24647201, 0.23877935, 0.24069646, ..., 0.2462533 , 0.25237438,
        0.24626514],
       [0.24037524, 0.248186  , 0.24073282, ..., 0.24341889, 0.23954508,
        0.23343675]])

In [23]:
# Assigning reconstructed matrix to df
pred_df = pd.DataFrame(pred_rating_n, columns = pivot.columns, index=reviewer_id).transpose()
pred_df.head(10)

,A0020356UF96ZV361ST,A00463782V7TKAP9EMNL,A0099735VDZ3HDCAAYKL,A103KWX1QFCG6I,A103M7E0BSFC83,A1098Z3D7ENJ2F,A10DX94ATZQEFG,A10ED33F8VY4GU,A10JAUCIGVRW9F,A10PEXB6XAQ5XF,A10RH00JS3DUX0,A10SXQMQBJ2L,A10WR0ZWDHWYI4,A10Y597K071WTQ,A1121LU5RSP5CH,A1168ILANAUMV8,A116GL1WSBL31P,A11B4CZH8ZB9QQ,A11KZJ5GD1TKC0,A11P853U6FIKAM,A11PLGX6L5KBYD,A11PRSJD8V0R2H,A11S2QYS4HV6SF,A128W553WN84EW,A12CZBKBM4OZMT,A12ED27FNHD0WY,A12K5Y2SEANYW0,A12NCJPL1Y3AH4,A12O5K3EQ4MC7Z,A12OXDSRRPOSPF,A12UMMBY3V99EK,A135NSTCX38DEI,A137UEIU93IFEF,A139X5E4RN7SUS,A13FGE5H3UXJLZ,A13J31ZJVVG2GY,A13JQT6036JF,A13LERNQ8R7267,A13Q7A1UWMNUU6,A13QONG1RD57UW,A13QTZ8CIMHHG4,A141H51I3H4B1S,A142HAT43L5W4W,A143IV9N6HPKKB,A14CRMGM3ZW3SP,A14DHYFGHKOYHZ,A14I311IIUMBWC,A14QJKY3DA19AP,A14R9XMZVJ6INB,A14TN4KQCZUV2N,...,AT8FSIABOPFMJ,ATTQEF12BAZNC,AUF0OPKICU6D4,AUFNC2SIHNZUN,AUI3XAZ1513SI,AUII0MVONK1A1,AUZ96COITGKE3,AV0DTTLGNW7C1,AV4HVQ5WUQ1Z1,AV6ORIR1JV86P,AV8X3WC4XB1WF,AVAC1JO1AZGAJ,AVGYENZU56KBR,AVJ8F6IFNLC4E,AVO3NDENWYHEH,AVSG37QWM774V,AVTN1DZPKSY8I,AW4NFCPN81CS3,AW7JJRAMYZY1X,AWANV5IBDLCBN,AWG9YC0S5ZXRQ,AWHX7AT8548LB,AWLFTOT8G8NJN,AWNYF3812RWNG,AWPBNX7NCVR0D,AX2OKB7GR55FZ,AX48LWJBU5RUH,AX5KL9LWC1TG6,AX5NHGPX6G44A,AXIO51VZN815F,AXM5ZEK9I7CWL,AXMNLUJHA9RG7,AXOY11IDEONHE,AXV7E488BQWYW,AXVDFGI9OEIVK,AXX9S5UGOEZR9,AY17DZYGNYJ0L,AY9AIGZKJYC6T,AYDMB9RBJMTBR,AYEO4YRBLBJ3N,AYGI3FN2M078F,AYIPGMUURK82V,AYJGHZSEI1J2N,AYRW797B5QB24,AYUTCGVSM1H7T,AYVJY4Z9CYQWU,AZGDIB6UL3QNL,AZMMFZKJB8PV6,AZPNSCZH6LW40,AZYERRDY2VW61
asin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
B003FL6X52,0.240799,0.241042,0.250659,0.246472,0.240375,0.244386,0.256805,0.241119,0.237331,0.240843,0.242496,0.243702,0.248528,0.255146,0.243231,0.245861,0.238658,0.252187,0.247277,0.243364,0.240926,0.241079,0.251797,0.241450,0.237628,0.240983,0.241436,0.241194,0.243380,0.266345,0.244521,0.245947,0.238526,0.247153,0.262575,0.242821,0.240174,0.247712,0.240677,0.239702,0.247981,0.247576,0.260926,0.241144,0.241247,0.241210,0.243424,0.240752,0.241205,0.242023,...,0.244925,0.235824,0.244049,0.229270,0.241611,0.246231,0.238102,0.244539,0.259227,0.247689,0.251121,0.255405,0.251532,0.248566,0.243041,0.238346,0.263968,0.250763,0.239510,0.243812,0.248737,0.248558,0.249207,0.234452,0.253216,0.243995,0.240708,0.240933,0.240653,0.240854,0.251536,0.228486,0.245093,0.239848,0.237515,0.252670,0.232371,0.236772,0.240819,0.255600,0.241188,0.244759,0.242135,0.240797,0.240597,0.235845,0.251465,0.256306,0.264066,0.242806
B003JTHMO6,0.242138,0.241117,0.234577,0.238779,0.248186,0.248473,0.250640,0.240808,0.290505,0.241533,0.243162,0.236423,0.237382,0.292754,0.238078,0.248146,0.238312,0.272868,0.246073,0.272328,0.264005,0.244902,0.234406,0.261564,0.243619,0.240996,0.247259,0.243859,0.240472,0.274482,0.239349,0.248727,0.278691,0.243267,0.251952,0.237622,0.239934,0.239634,0.240755,0.234235,0.249128,0.276456,0.238169,0.248348,0.245617,0.243027,0.248813,0.241542,0.241227,0.241959,...,0.235362,0.250058,0.243665,0.244906,0.248794,0.247783,0.232126,0.244111,0.234971,0.243076,0.242053,0.243167,0.255402,0.262524,0.247527,0.240528,0.243045,0.245509,0.241771,0.245223,0.238434,0.228481,0.248132,0.291440,0.248605,0.241192,0.240848,0.249729,0.241486,0.244340,0.242300,0.240253,0.243837,0.243404,0.243338,0.241916,0.241526,0.231512,0.241003,0.297665,0.236527,0.239235,0.238672,0.265487,0.255619,0.297689,0.230879,0.254304,0.238716,0.294821
B003P2QCE8,0.240674,0.240679,0.240708,0.240696,0.240733,0.240524,0.240557,0.240682,0.240721,0.240690,0.240674,0.240712,0.240662,0.240883,0.240745,0.240675,0.240660,0.240681,0.240659,0.240769,0.240519,0.240705,0.240681,0.240703,0.240814,0.240678,0.240743,0.240665,0.240696,0.240575,0.240626,0.240644,0.240781,0.240699,0.240683,0.240671,0.240669,0.240731,0.240680,0.240735,0.240655,0.240802,0.240680,0.240563,0.240659,0.240644,0.240652,0.240685,0.240676,0.240664,...,0.240685,0.240702,0.240673,0.240633,0.240667,0.240645,0.240670,0.240694,0.240643,0.240725,0.240697,0.240751,0.240725,0.240747,0.240

Now, I can have the SVD latent factorization of each user for each item. It is time for recommendation function.

In [29]:
def recommend_items(user_id ,pred_df, items_df, items_to_ignore=[], top_list=20, verbose=False):
        '''this function find the most related items'''
        # taking and sorting the user's predictions
        sorted_user_predictions = pred_df[user_id].sort_values(ascending=False) \
                                    .reset_index().rename(columns={user_id: 'recStrength'})

        recommendations_df = sorted_user_predictions[~sorted_user_predictions['asin'].isin(items_to_ignore)] \
                               .sort_values('recStrength', ascending = False) \
                               .head(top_list)

        return recommendations_df
    


'Recstrength' shows the how much our prediction is strength.

In [32]:
df_meta = pd.read_csv('kindle_meta_last.csv', low_memory=False) #getting metadata for title names

In [45]:
def recommender(user_id,pred_df,real_df,df_meta):
    '''this functions merges the title with corresponding book id and finding the 
        empty rows which means does not read by user and priting results'''
        
    recommend = recommend_items(user_id, pred_df,real_df)
    df_user= real_df.loc[real_df['reviewerID'] == user_id]
    new_df = df_user.merge(recommend, how = 'outer', left_on = 'asin', right_on = 'asin')
    rec_df = new_df.loc[new_df['overall'].isnull()==True]
    df_rec_t = rec_df.loc[:, ['asin', 'recStrength']]
    df_last = pd.merge(df_rec_t,df_meta[['title','asin']],on=['asin'], how='left') 
    return df_last

# Getting Recommendations 

When I try my recommender system it gives me recommended books. Maybe strength looks not well, but it is expected because I am working now only the small subset of original data. My aim is to build a system, if systems works for this subset, it can be run in more strong computers.

In [35]:
recommender('A0099735VDZ3HDCAAYKL',pred_df,df_1000,df_meta)

,asin,recStrength,title
0,B017AJ4QFI,0.473874,A Very Alpha Christmas: Over 25 Paranormal Hol...
1,B0135MRXRM,0.473195,Sleeping BBW And The Billionaire Bear (The Shi...
2,B013L4T8Y0,0.473124,Three Grizzlies Gruff (Bear-y Spicy Fairy Tale...
3,B010TYBN4S,0.461972,Wanting a Mate (BBW Paranormal Shapeshifter Ro...
4,B010ZFRUQQ,0.454028,Goldilocks And The Three Bear Shifters (Bear-y...
5,B014B6KG2K,0.451822,Rumpled Bear Skin (Seattle&#39;s Billionaire B...
6,B01CSNZ15K,0.438699,Chief Bear (Rescue Bears Book 1) - Kindle edition
7,B013X96TL8,0.437376,Rapunzel And The Billionaire Bear (The Shifter...
8,B00UGP39S4,0.430844,Furbidden: BBW Paranormal Shape Shifter Romanc...
9,B0130LQWBG,0.426664,Hunting a Mate (BBW Paranormal Shapeshifter Ro...


# Another Technique

Now, I will use cosine similarity tool of scikit learn library directly and build a user-based recommender.

In [46]:
user_sparse_pivot = sparse.csr_matrix(pivot.fillna(0))
user_recommender = pw.cosine_similarity(user_sparse_pivot)

In [47]:
# normalizing zero values 
pred_rating_n_cos = (user_recommender - user_recommender.mean())/ (user_recommender.max() - user_recommender.min())

In [48]:
user_recommender_df = pd.DataFrame(pred_rating_n_cos, columns=pivot.index.values,index = pivot.index.values)

In this method, I have user-user matrix, which shows relations of them, now I will change it to get recommendations.

In [356]:
user_recommender_df.head(3)

,A0020356UF96ZV361ST,A00463782V7TKAP9EMNL,A0099735VDZ3HDCAAYKL,A103KWX1QFCG6I,A103M7E0BSFC83,A1098Z3D7ENJ2F,A10DX94ATZQEFG,A10ED33F8VY4GU,A10JAUCIGVRW9F,A10PEXB6XAQ5XF,A10RH00JS3DUX0,A10SXQMQBJ2L,A10WR0ZWDHWYI4,A10Y597K071WTQ,A1121LU5RSP5CH,A1168ILANAUMV8,A116GL1WSBL31P,A11B4CZH8ZB9QQ,A11KZJ5GD1TKC0,A11P853U6FIKAM,A11PLGX6L5KBYD,A11PRSJD8V0R2H,A11S2QYS4HV6SF,A128W553WN84EW,A12CZBKBM4OZMT,A12ED27FNHD0WY,A12K5Y2SEANYW0,A12NCJPL1Y3AH4,A12O5K3EQ4MC7Z,A12OXDSRRPOSPF,A12UMMBY3V99EK,A135NSTCX38DEI,A137UEIU93IFEF,A139X5E4RN7SUS,A13FGE5H3UXJLZ,A13J31ZJVVG2GY,A13JQT6036JF,A13LERNQ8R7267,A13Q7A1UWMNUU6,A13QONG1RD57UW,A13QTZ8CIMHHG4,A141H51I3H4B1S,A142HAT43L5W4W,A143IV9N6HPKKB,A14CRMGM3ZW3SP,A14DHYFGHKOYHZ,A14I311IIUMBWC,A14QJKY3DA19AP,A14R9XMZVJ6INB,A14TN4KQCZUV2N,...,AT8FSIABOPFMJ,ATTQEF12BAZNC,AUF0OPKICU6D4,AUFNC2SIHNZUN,AUI3XAZ1513SI,AUII0MVONK1A1,AUZ96COITGKE3,AV0DTTLGNW7C1,AV4HVQ5WUQ1Z1,AV6ORIR1JV86P,AV8X3WC4XB1WF,AVAC1JO1AZGAJ,AVGYENZU56KBR,AVJ8F6IFNLC4E,AVO3NDENWYHEH,AVSG37QWM774V,AVTN1DZPKSY8I,AW4NFCPN81CS3,AW7JJRAMYZY1X,AWANV5IBDLCBN,AWG9YC0S5ZXRQ,AWHX7AT8548LB,AWLFTOT8G8NJN,AWNYF3812RWNG,AWPBNX7NCVR0D,AX2OKB7GR55FZ,AX48LWJBU5RUH,AX5KL9LWC1TG6,AX5NHGPX6G44A,AXIO51VZN815F,AXM5ZEK9I7CWL,AXMNLUJHA9RG7,AXOY11IDEONHE,AXV7E488BQWYW,AXVDFGI9OEIVK,AXX9S5UGOEZR9,AY17DZYGNYJ0L,AY9AIGZKJYC6T,AYDMB9RBJMTBR,AYEO4YRBLBJ3N,AYGI3FN2M078F,AYIPGMUURK82V,AYJGHZSEI1J2N,AYRW797B5QB24,AYUTCGVSM1H7T,AYVJY4Z9CYQWU,AZGDIB6UL3QNL,AZMMFZKJB8PV6,AZPNSCZH6LW40,AZYERRDY2VW61
A0020356UF96ZV361ST,0.972044,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,0.325943,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,0.266231,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,...,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,0.170051,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956
A00463782V7TKAP9EMNL,-0.027956,0.972044,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,0.362288,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,0.291980,-0.027956,...,-0.027956,-0.027956,0.262521,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956
A0099735VDZ3HDCAAYKL,-0.027956,-0.027956,0.972044,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,0.042509,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,0.195763,-0.000096,0.128169,-0.027956,-0.027956,-0.027956,0.253957,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,0.008896,-0.027956,0.203908,0.011139,0.091589,-0.027956,-0.027956,-0.027956,-0.027956,0.002775,-0.002138,-0.027956,-0.027956,-0.027956,-0.027956,-0.027956,

In [50]:
def recommend2(user_recommender_df,user_id,df_meta):
    '''This function finds the top-2 similar users first,
    then it takes the mean of their ratings to books and sorted books according to mean values of books
    '''
    user_cosine_df = pd.DataFrame(user_recommender_df[user_id].sort_values(ascending=False))
    user_cosine_df.reset_index(level=0, inplace=True)
    user_cosine_df.columns = ['asin','cosine_sim']
    
    # for 2-similar users
    similar_user = list(user_cosine_df['asin'][1:3].values)
    similar_user_df = pivot.T[[user_id] + similar_user]
    similar_user_df['mean'] = similar_user_df[similar_user].mean(numeric_only=True,axis=1)
    similar_user_df.sort_values('mean', ascending=False,inplace = True)
    
    #taking top 10 books from similar users and merging them
    book_top10 = similar_user_df[similar_user_df[user_id]==0].head(10)
    df_last1 = pd.merge(book_top10,df_meta[['title','asin']],on=['asin'], how='left') 
    return df_last1

In [51]:
recommend2(user_recommender_df,'A0099735VDZ3HDCAAYKL',df_meta)

,asin,A0099735VDZ3HDCAAYKL,A1BOINGLPB5I1M,A1V0IK7DUYIVAC,mean,title
0,B00LNJ4ABC,0.0,5.0,5.0,5.0,Kodiak&#39;s Claim (Kodiak Point Book 1) - Kin...
1,B00YQ3UDM2,0.0,5.0,5.0,5.0,Double Dragons: BBW Paranormal Romance (Dragon...
2,B00QL65B2Y,0.0,5.0,5.0,5.0,Separation Zone (Immortal Ops Book 7) - Kindle...
3,B015BRVKQO,0.0,5.0,5.0,5.0,Moon&#39;s Law (New Moon Wolves 2) - Kindle ed...
4,B0165SOAPG,0.0,5.0,5.0,5.0,Jace (Riverwise Private Security 2) - Wolf Shi...
5,B00F9Z458E,0.0,0.0,5.0,2.5,The Draglen Brothers - DRAKEN (BK 1) - Kindle ...
6,B01DGXSHRK,0.0,5.0,0.0,2.5,Barbarian&#39;s Mate
7,B00JZWDC9K,0.0,0.0,5.0,2.5,King of Prey: (A Bird Shifter Novel) - Kindle ...
8,B00PD8LQ6G,0.0,5.0,0.0,2.5,Ready to Were: Shift Happens Series Book One -...
9,B015WOU89Q,0.0,5.0,0.0,2.5,Wild Bear (Bear Haven Book 3) - Kindle edition


When we look at these 2 different recommendations, it is shown that they recommends different books. When we change our system, results have changed. There are many ways to compare results in recommendation systems. As a future plan, I will add comparing metric to my systems. 

To get idea, I used these resources in this notebook:
    
- https://medium.com/@bkexcel2014/building-movie-recommender-systems-using-cosine-similarity-in-python-eff2d4e60d24
- https://www.kaggle.com/gspmoreira/recommender-systems-in-python-101
- https://towardsdatascience.com/various-implementations-of-collaborative-filtering-100385c6dfe0